In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.sparse import lil_matrix, csr_matrix
from collections import defaultdict
import time
from datetime import datetime
import pandas as pd


# Carga de Datos

In [2]:
direccion_B2B_transacciones = '../Datos/base_3_transaccional_b2b.txt'
df_b2b = pd.read_csv(direccion_B2B_transacciones, sep="\t", encoding='utf-8')
df_b2b['fecha'] = pd.to_datetime(df_b2b['fecha_factura'])


## Análisis inicial de datos

In [3]:
# Explorar la estructura de datos
print("=== ANÁLISIS INICIAL DE DATOS B2B ===")
print(f"Shape del dataset: {df_b2b.shape}")
print(f"Periodo: {df_b2b['fecha'].min()} a {df_b2b['fecha'].max()}")
print(f"\nClientes únicos: {df_b2b['id_b2b'].nunique():,}")
print(f"Productos únicos: {df_b2b['producto'].nunique():,}")
print(f"Municipios únicos: {df_b2b['municipio'].nunique():,}")
print(f"Zonas únicas: {df_b2b['zona'].nunique():,}")

print(f"\nValor total min: ${df_b2b['valor_total'].min():,.0f}")
print(f"Valor total max: ${df_b2b['valor_total'].max():,.0f}")
print(f"Valor total promedio: ${df_b2b['valor_total'].mean():,.0f}")

# Ver distribución de alineación estratégica
print(f"\nDistribución alineación estratégica:")
print(df_b2b['alineación con portafolio estratégico b2b'].describe())

=== ANÁLISIS INICIAL DE DATOS B2B ===
Shape del dataset: (25866, 11)
Periodo: 2007-01-03 00:00:00 a 2010-02-18 00:00:00

Clientes únicos: 6
Productos únicos: 2,564
Municipios únicos: 5
Zonas únicas: 3

Valor total min: $5
Valor total max: $68,752
Valor total promedio: $1,536

Distribución alineación estratégica:
count    25866.000000
mean         0.000294
std          0.000155
min         -0.008032
25%          0.000225
50%          0.000292
75%          0.000371
max          0.000654
Name: alineación con portafolio estratégico b2b, dtype: float64


## Features de Valor del Cliente

In [4]:

# 1. Métricas agregadas por cliente
client_metrics = df_b2b.groupby('id_b2b').agg({
    'valor_total': ['sum', 'mean', 'count', 'std'],
    'alineación con portafolio estratégico b2b': ['mean', 'sum'],
    'fecha': ['min', 'max'],
    'producto': 'nunique',
    'zona': 'nunique',
    'categoria_b2b_macro': 'nunique'
}).round(2)

# Aplanar nombres de columnas
client_metrics.columns = ['_'.join(col).strip() for col in client_metrics.columns]
client_metrics = client_metrics.reset_index()

# Renombrar columnas para claridad
client_metrics.rename(columns={
    'valor_total_sum': 'valor_total_cliente',
    'valor_total_mean': 'valor_promedio_transaccion',
    'valor_total_count': 'n_transacciones_cliente',
    'valor_total_std': 'volatilidad_gasto',
    'alineación con portafolio estratégico b2b_mean': 'alineacion_promedio_cliente',
    'alineación con portafolio estratégico b2b_sum': 'alineacion_total_cliente',
    'fecha_min': 'primera_compra',
    'fecha_max': 'ultima_compra',
    'producto_nunique': 'productos_distintos_cliente',
    'zona_nunique': 'zonas_distintas_cliente',
    'categoria_b2b_macro_nunique': 'categorias_distintas_cliente'
}, inplace=True)

# 2. Calcular días activo y frecuencia
client_metrics['dias_activo'] = (client_metrics['ultima_compra'] - client_metrics['primera_compra']).dt.days + 1
client_metrics['frecuencia_compra'] = client_metrics['n_transacciones_cliente'] / client_metrics['dias_activo'] * 30  # transacciones por mes

# 3. Calcular percentiles de valor
client_metrics['valor_percentile'] = pd.qcut(client_metrics['valor_total_cliente'], 
                                           q=10, labels=False, duplicates='drop') + 1

# 4. Segmentación de clientes por valor
def categorizar_cliente_valor(percentile):
    if percentile >= 8:
        return 'Alto_Valor'
    elif percentile >= 5:
        return 'Medio_Valor'
    else:
        return 'Bajo_Valor'

client_metrics['segmento_valor'] = client_metrics['valor_percentile'].apply(categorizar_cliente_valor)

print(f"Features de cliente creadas: {client_metrics.shape}")
print(f"Distribución por segmento:")
print(client_metrics['segmento_valor'].value_counts())

Features de cliente creadas: (6, 16)
Distribución por segmento:
segmento_valor
Bajo_Valor     3
Alto_Valor     2
Medio_Valor    1
Name: count, dtype: int64


## Features de Reposición y Temporales

In [5]:
print("=== CREANDO FEATURES DE REPOSICIÓN Y TEMPORALES ===")

# 1. Añadir features temporales básicas
df_b2b['año'] = df_b2b['fecha'].dt.year
df_b2b['mes'] = df_b2b['fecha'].dt.month
df_b2b['trimestre'] = df_b2b['fecha'].dt.quarter
df_b2b['dia_semana'] = df_b2b['fecha'].dt.dayofweek
df_b2b['semana_año'] = df_b2b['fecha'].dt.isocalendar().week

# 2. Análisis de reposición por cliente-producto
cliente_producto_stats = df_b2b.groupby(['id_b2b', 'producto']).agg({
    'fecha': ['count', 'min', 'max'],
    'valor_total': ['mean', 'sum', 'std'],
    'alineación con portafolio estratégico b2b': 'mean'
}).round(2)

cliente_producto_stats.columns = ['_'.join(col).strip() for col in cliente_producto_stats.columns]
cliente_producto_stats = cliente_producto_stats.reset_index()

cliente_producto_stats.rename(columns={
    'fecha_count': 'veces_comprado',
    'fecha_min': 'primera_compra_producto',
    'fecha_max': 'ultima_compra_producto',
    'valor_total_mean': 'valor_promedio_producto',
    'valor_total_sum': 'valor_total_producto',
    'valor_total_std': 'volatilidad_producto',
    'alineación con portafolio estratégico b2b_mean': 'alineacion_producto'
}, inplace=True)

# 3. Calcular ciclos de reposición
cliente_producto_stats['dias_entre_compras'] = (
    cliente_producto_stats['ultima_compra_producto'] - 
    cliente_producto_stats['primera_compra_producto']
).dt.days

# Solo para productos comprados más de una vez
mask_multiple = cliente_producto_stats['veces_comprado'] > 1
cliente_producto_stats.loc[mask_multiple, 'ciclo_reposicion_dias'] = (
    cliente_producto_stats.loc[mask_multiple, 'dias_entre_compras'] / 
    (cliente_producto_stats.loc[mask_multiple, 'veces_comprado'] - 1)
)

# 4. Días desde última compra (calculado desde la fecha máxima en datos)
fecha_referencia = df_b2b['fecha'].max()
cliente_producto_stats['dias_desde_ultima_compra'] = (
    fecha_referencia - cliente_producto_stats['ultima_compra_producto']
).dt.days

print(f"Stats cliente-producto creadas: {cliente_producto_stats.shape}")
print(f"Productos con ciclo de reposición identificado: {cliente_producto_stats['ciclo_reposicion_dias'].notna().sum():,}")

=== CREANDO FEATURES DE REPOSICIÓN Y TEMPORALES ===
Stats cliente-producto creadas: (4885, 12)
Productos con ciclo de reposición identificado: 2,760


## Features Geográficas y de Popularidad

In [6]:
print("=== CREANDO FEATURES GEOGRÁFICAS Y DE POPULARIDAD ===")

# 1. Popularidad de productos por zona
producto_zona_stats = df_b2b.groupby(['zona', 'producto']).agg({
    'valor_total': ['sum', 'count', 'mean'],
    'id_b2b': 'nunique'
}).round(2)

producto_zona_stats.columns = ['_'.join(col).strip() for col in producto_zona_stats.columns]
producto_zona_stats = producto_zona_stats.reset_index()
producto_zona_stats.rename(columns={
    'valor_total_sum': 'valor_total_zona_producto',
    'valor_total_count': 'transacciones_zona_producto', 
    'valor_total_mean': 'valor_promedio_zona_producto',
    'id_b2b_nunique': 'clientes_unicos_zona_producto'
}, inplace=True)

# 2. Popularidad de productos por municipio
producto_municipio_stats = df_b2b.groupby(['municipio', 'producto']).agg({
    'valor_total': ['sum', 'count'],
    'id_b2b': 'nunique'
}).round(2)

producto_municipio_stats.columns = ['_'.join(col).strip() for col in producto_municipio_stats.columns]
producto_municipio_stats = producto_municipio_stats.reset_index()
producto_municipio_stats.rename(columns={
    'valor_total_sum': 'valor_total_municipio_producto',
    'valor_total_count': 'transacciones_municipio_producto',
    'id_b2b_nunique': 'clientes_unicos_municipio_producto'
}, inplace=True)

# 3. Métricas agregadas por zona
zona_stats = df_b2b.groupby('zona').agg({
    'valor_total': ['sum', 'mean', 'count'],
    'id_b2b': 'nunique',
    'producto': 'nunique'
}).round(2)

zona_stats.columns = ['_'.join(col).strip() for col in zona_stats.columns]
zona_stats = zona_stats.reset_index()
zona_stats.rename(columns={
    'valor_total_sum': 'valor_total_zona',
    'valor_total_mean': 'valor_promedio_zona',
    'valor_total_count': 'transacciones_zona',
    'id_b2b_nunique': 'clientes_zona',
    'producto_nunique': 'productos_zona'
}, inplace=True)

# 4. Ranking de productos por zona
producto_zona_stats['rank_producto_en_zona'] = producto_zona_stats.groupby('zona')['valor_total_zona_producto'].rank(ascending=False)

print(f"Stats zona-producto: {producto_zona_stats.shape}")
print(f"Stats municipio-producto: {producto_municipio_stats.shape}")
print(f"Stats por zona: {zona_stats.shape}")

=== CREANDO FEATURES GEOGRÁFICAS Y DE POPULARIDAD ===
Stats zona-producto: (3852, 7)
Stats municipio-producto: (4527, 5)
Stats por zona: (3, 6)


## Features de Co-ocurrencia (Cross-selling)

In [7]:
print("=== CREANDO FEATURES DE CO-OCURRENCIA ===")

# 1. Análisis de productos comprados juntos por cliente
# Crear un identificador de sesión de compra (mismo cliente, mismo mes)
df_b2b['periodo_compra'] = df_b2b['fecha'].dt.to_period('M')
df_b2b['sesion_compra'] = df_b2b['id_b2b'].astype(str) + '_' + df_b2b['periodo_compra'].astype(str)

# 2. Productos por sesión
productos_por_sesion = df_b2b.groupby('sesion_compra')['producto'].apply(list).reset_index()
productos_por_sesion['n_productos_sesion'] = productos_por_sesion['producto'].apply(len)

# 3. Co-ocurrencia básica (productos que aparecen juntos)
from itertools import combinations
co_ocurrencia = []

for _, row in productos_por_sesion.iterrows():
    productos = row['producto']
    if len(productos) > 1:  # Solo sesiones con más de un producto
        for p1, p2 in combinations(productos, 2):
            co_ocurrencia.append({
                'producto_1': p1,
                'producto_2': p2,
                'sesion': row['sesion_compra']
            })

co_ocurrencia_df = pd.DataFrame(co_ocurrencia)

# 4. Contar frecuencias de co-ocurrencia
if len(co_ocurrencia_df) > 0:
    co_frequency = co_ocurrencia_df.groupby(['producto_1', 'producto_2']).size().reset_index(name='frecuencia_co_compra')
    co_frequency['producto_par'] = co_frequency['producto_1'] + '_' + co_frequency['producto_2']
    
    # Top 20 pares más frecuentes
    top_pairs = co_frequency.nlargest(20, 'frecuencia_co_compra')
    print("Top 10 pares de productos más co-comprados:")
    print(top_pairs.head(10)[['producto_1', 'producto_2', 'frecuencia_co_compra']])
else:
    print("No se encontraron co-ocurrencias suficientes")
    co_frequency = pd.DataFrame()

print(f"Sesiones de compra identificadas: {len(productos_por_sesion):,}")
print(f"Pares de co-ocurrencia encontrados: {len(co_frequency):,}")

=== CREANDO FEATURES DE CO-OCURRENCIA ===
Top 10 pares de productos más co-comprados:
          producto_1    producto_2  frecuencia_co_compra
741203  Producto_377  Producto_377                   469
740327  Producto_377  Producto_167                   379
344683  Producto_167  Producto_377                   337
832168   Producto_54  Producto_377                   283
839987   Producto_55  Producto_377                   280
741421  Producto_377   Producto_61                   272
942419   Producto_73  Producto_377                   250
847467   Producto_56  Producto_377                   246
740496  Producto_377  Producto_189                   242
855698   Producto_57  Producto_377                   234
Sesiones de compra identificadas: 159
Pares de co-ocurrencia encontrados: 1,156,241


## Consolidación de Features y Preparación de Datos

In [8]:
print("=== CONSOLIDANDO FEATURES PARA ALGORITMO B2B ===")

# 1. Mergear todas las features en el dataset principal
df_b2b_enhanced = df_b2b.merge(client_metrics[['id_b2b', 'valor_total_cliente', 'segmento_valor', 
                                              'valor_promedio_transaccion', 'frecuencia_compra']], 
                               on='id_b2b', how='left')

# 2. Añadir features de zona
df_b2b_enhanced = df_b2b_enhanced.merge(zona_stats[['zona', 'valor_total_zona', 'productos_zona']], 
                                       on='zona', how='left')

# 3. Añadir features de producto-zona
df_b2b_enhanced = df_b2b_enhanced.merge(producto_zona_stats[['zona', 'producto', 'rank_producto_en_zona', 
                                                           'valor_total_zona_producto']], 
                                       on=['zona', 'producto'], how='left')

# 4. Crear features derivadas para el algoritmo
df_b2b_enhanced['valor_relativo_cliente'] = df_b2b_enhanced['valor_total'] / df_b2b_enhanced['valor_promedio_transaccion']
df_b2b_enhanced['score_valor_estrategico'] = (
    df_b2b_enhanced['valor_total'] * 
    df_b2b_enhanced['alineación con portafolio estratégico b2b']
)

# 5. Llenar valores faltantes
df_b2b_enhanced['rank_producto_en_zona'] = df_b2b_enhanced['rank_producto_en_zona'].fillna(999)
df_b2b_enhanced['valor_total_zona_producto'] = df_b2b_enhanced['valor_total_zona_producto'].fillna(0)

print(f"Dataset enriquecido creado: {df_b2b_enhanced.shape}")
print(f"Columnas disponibles: {len(df_b2b_enhanced.columns)}")
print("\nPrimeras columnas agregadas:")
print(df_b2b_enhanced[['id_b2b', 'producto', 'segmento_valor', 'valor_relativo_cliente', 'score_valor_estrategico']].head())

=== CONSOLIDANDO FEATURES PARA ALGORITMO B2B ===
Dataset enriquecido creado: (25866, 28)
Columnas disponibles: 28

Primeras columnas agregadas:
   id_b2b     producto segmento_valor  valor_relativo_cliente  \
0  B2B_01   Producto_1    Medio_Valor                2.034511   
1  B2B_01   Producto_2    Medio_Valor                1.915374   
2  B2B_02   Producto_4     Alto_Valor                0.159161   
3  B2B_02  Producto_13     Alto_Valor                2.533815   
4  B2B_02  Producto_14     Alto_Valor                3.775159   

   score_valor_estrategico  
0                 0.392598  
1                 0.131342  
2                 0.014338  
3                 1.022888  
4                 0.542556  


## Motor de Maximización de Valor (Componente 1)

In [9]:
print("=== IMPLEMENTANDO MOTOR DE MAXIMIZACIÓN DE VALOR ===")

def calculate_value_score(df, client_id, product_id, weight=0.4):
    """
    Calcula score de valor para un producto dado un cliente
    """
    # Obtener información del cliente
    client_info = df[df['id_b2b'] == client_id].iloc[0] if len(df[df['id_b2b'] == client_id]) > 0 else None
    
    if client_info is None:
        return 0
    
    # Obtener información del producto
    product_info = df[df['producto'] == product_id]
    
    if len(product_info) == 0:
        return 0
    
    # Métricas del producto
    avg_value = product_info['valor_total'].mean()
    avg_alignment = product_info['alineación con portafolio estratégico b2b'].mean()
    
    # Score base: valor promedio del producto * alineación estratégica
    base_score = avg_value * avg_alignment
    
    # Ajuste por segmento del cliente
    segment_multiplier = {
        'Alto_Valor': 1.3,
        'Medio_Valor': 1.1,
        'Bajo_Valor': 0.9
    }
    
    client_segment = client_info['segmento_valor']
    multiplier = segment_multiplier.get(client_segment, 1.0)
    
    final_score = base_score * multiplier * weight
    
    return final_score

def get_value_recommendations(df, client_id, n_recommendations=10):
    """
    Genera recomendaciones basadas en maximización de valor
    """
    # Obtener todos los productos únicos
    all_products = df['producto'].unique()
    
    # Calcular scores para todos los productos
    product_scores = []
    for product in all_products:
        score = calculate_value_score(df, client_id, product)
        if score > 0:
            product_scores.append({
                'producto': product,
                'value_score': score
            })
    
    # Convertir a DataFrame y ordenar
    recommendations_df = pd.DataFrame(product_scores)
    if len(recommendations_df) > 0:
        recommendations_df = recommendations_df.sort_values('value_score', ascending=False).head(n_recommendations)
    
    return recommendations_df

# Prueba con un cliente
test_client = df_b2b_enhanced['id_b2b'].unique()[0]
value_recs = get_value_recommendations(df_b2b_enhanced, test_client, 5)
print(f"Recomendaciones de valor para cliente {test_client}:")
print(value_recs)


=== IMPLEMENTANDO MOTOR DE MAXIMIZACIÓN DE VALOR ===
Recomendaciones de valor para cliente B2B_01:
           producto  value_score
891    Producto_983     4.151230
1146  Producto_1247     3.767888
1052  Producto_1151     2.924689
1147  Producto_1248     2.757262
890    Producto_982     2.427475


## Predictor de Reposición Inteligente (Componente 2)

In [10]:
print("=== IMPLEMENTANDO PREDICTOR DE REPOSICIÓN ===")

def calculate_replenishment_urgency(client_product_stats, client_id, product_id, weight=0.3):
    """
    Calcula urgencia de reposición para un producto específico
    """
    # Buscar estadísticas del cliente-producto
    client_product = client_product_stats[
        (client_product_stats['id_b2b'] == client_id) & 
        (client_product_stats['producto'] == product_id)
    ]
    
    if len(client_product) == 0:
        return 0
    
    stats = client_product.iloc[0]
    
    # Si nunca ha comprado este producto, score = 0
    if pd.isna(stats['dias_desde_ultima_compra']):
        return 0
    
    # Si solo lo compró una vez, usar promedio de la industria (60 días)
    if pd.isna(stats['ciclo_reposicion_dias']):
        expected_cycle = 60
    else:
        expected_cycle = stats['ciclo_reposicion_dias']
    
    days_since_last = stats['dias_desde_ultima_compra']
    
    # Calcular urgencia: cuanto más cerca del ciclo esperado, mayor urgencia
    if expected_cycle <= 0:
        return 0
    
    urgency_ratio = days_since_last / expected_cycle
    
    # Score máximo cuando está en 80%-120% del ciclo esperado
    if 0.8 <= urgency_ratio <= 1.2:
        urgency_score = 1.0
    elif urgency_ratio > 1.2:
        # Muy tarde, alta urgencia pero decae con el tiempo
        urgency_score = max(0.5, 2.0 - urgency_ratio)
    else:
        # Muy temprano, baja urgencia
        urgency_score = urgency_ratio / 0.8
    
    # Ajustar por frecuencia de compra histórica
    frequency_bonus = min(1.5, stats['veces_comprado'] / 5.0)
    
    final_score = urgency_score * frequency_bonus * weight
    
    return final_score

def get_replenishment_recommendations(df, client_product_stats, client_id, n_recommendations=10):
    """
    Genera recomendaciones basadas en ciclos de reposición
    """
    # Obtener productos que el cliente ha comprado antes
    client_products = client_product_stats[
        client_product_stats['id_b2b'] == client_id
    ]['producto'].unique()
    
    product_scores = []
    for product in client_products:
        score = calculate_replenishment_urgency(client_product_stats, client_id, product)
        if score > 0:
            product_scores.append({
                'producto': product,
                'replenishment_score': score
            })
    
    recommendations_df = pd.DataFrame(product_scores)
    if len(recommendations_df) > 0:
        recommendations_df = recommendations_df.sort_values('replenishment_score', ascending=False).head(n_recommendations)
    
    return recommendations_df

# Prueba del predictor de reposición
replenishment_recs = get_replenishment_recommendations(df_b2b_enhanced, cliente_producto_stats, test_client, 5)
print(f"Recomendaciones de reposición para cliente {test_client}:")
print(replenishment_recs)

=== IMPLEMENTANDO PREDICTOR DE REPOSICIÓN ===
Recomendaciones de reposición para cliente B2B_01:
         producto  replenishment_score
543   Producto_54                 0.45
480  Producto_377                 0.45
586   Producto_68                 0.45
474  Producto_366                 0.45
584   Producto_67                 0.45


## Maximizador de Cross-Selling Geográfico (Componente 3)

In [11]:
print("=== IMPLEMENTANDO CROSS-SELLING GEOGRÁFICO ===")

def calculate_geographic_score(df, product_zona_stats, client_id, product_id, weight=0.2):
    """
    Calcula score geográfico basado en popularidad en la zona del cliente
    """
    # Obtener zona del cliente
    client_zone = df[df['id_b2b'] == client_id]['zona'].iloc[0] if len(df[df['id_b2b'] == client_id]) > 0 else None
    
    if client_zone is None:
        return 0
    
    # Buscar estadísticas del producto en esa zona
    zone_product = product_zona_stats[
        (product_zona_stats['zona'] == client_zone) & 
        (product_zona_stats['producto'] == product_id)
    ]
    
    if len(zone_product) == 0:
        return 0
    
    stats = zone_product.iloc[0]
    
    # Score basado en ranking en la zona (mejor ranking = mayor score)
    rank = stats['rank_producto_en_zona']
    max_rank = product_zona_stats[product_zona_stats['zona'] == client_zone]['rank_producto_en_zona'].max()
    
    if max_rank <= 0:
        return 0
    
    # Normalizar ranking (1 = mejor, 0 = peor)
    normalized_rank = 1 - (rank - 1) / max_rank
    
    # Bonus por valor total en la zona
    valor_bonus = min(2.0, stats['valor_total_zona_producto'] / 1000000)  # Normalizar por millón
    
    final_score = normalized_rank * valor_bonus * weight
    
    return final_score

def get_geographic_recommendations(df, product_zona_stats, client_id, n_recommendations=10):
    """
    Genera recomendaciones basadas en popularidad geográfica
    """
    # Obtener zona del cliente
    client_zone = df[df['id_b2b'] == client_id]['zona'].iloc[0] if len(df[df['id_b2b'] == client_id]) > 0 else None
    
    if client_zone is None:
        return pd.DataFrame()
    
    # Obtener productos populares en esa zona
    zone_products = product_zona_stats[product_zona_stats['zona'] == client_zone]['producto'].unique()
    
    product_scores = []
    for product in zone_products:
        score = calculate_geographic_score(df, product_zona_stats, client_id, product)
        if score > 0:
            product_scores.append({
                'producto': product,
                'geographic_score': score
            })
    
    recommendations_df = pd.DataFrame(product_scores)
    if len(recommendations_df) > 0:
        recommendations_df = recommendations_df.sort_values('geographic_score', ascending=False).head(n_recommendations)
    
    return recommendations_df

# Prueba del cross-selling geográfico
geographic_recs = get_geographic_recommendations(df_b2b_enhanced, producto_zona_stats, test_client, 5)
print(f"Recomendaciones geográficas para cliente {test_client}:")
print(geographic_recs)

=== IMPLEMENTANDO CROSS-SELLING GEOGRÁFICO ===
Recomendaciones geográficas para cliente B2B_01:
          producto  geographic_score
1547  Producto_377          0.195109
610   Producto_167          0.059718
1598  Producto_438          0.055995
1321  Producto_251          0.054715
1792  Producto_721          0.053717


## Algoritmo de Co-ocurrencia Inteligente

In [12]:
def build_cooccurrence_matrix_optimized(df, co_frequency):
    """
    Versión optimizada que precalcula valores promedio
    """
    from scipy.sparse import lil_matrix
    import numpy as np
    
    print("=== VERSIÓN OPTIMIZADA ===")
    print("Precalculando valores promedio por producto...")
    
    # Precalcular valores promedio por producto para evitar repetir cálculos
    product_avg_values = df.groupby('producto')['valor_total'].mean().to_dict()
    print(f"Valores promedio calculados para {len(product_avg_values):,} productos")
    
    # Resto igual pero usando el diccionario precalculado
    all_products = df['producto'].unique()
    product_to_idx = {prod: idx for idx, prod in enumerate(sorted(all_products))}
    idx_to_product = {idx: prod for prod, idx in product_to_idx.items()}
    
    n_products = len(all_products)
    cooccurrence_matrix = lil_matrix((n_products, n_products))
    
    total_pairs = len(co_frequency)
    print(f"Procesando {total_pairs:,} pares...")
    
    for i, row in co_frequency.iterrows():
        p1, p2 = row['producto_1'], row['producto_2']
        freq = row['frecuencia_co_compra']
        
        if p1 in product_to_idx and p2 in product_to_idx:
            idx1, idx2 = product_to_idx[p1], product_to_idx[p2]
            
            # Usar valores precalculados
            avg_value = (product_avg_values.get(p1, 0) + product_avg_values.get(p2, 0)) / 2
            score = freq * np.log1p(avg_value)
            
            cooccurrence_matrix[idx1, idx2] = score
            cooccurrence_matrix[idx2, idx1] = score
        
        if i % 10000 == 0:
            progress = (i / total_pairs) * 100
            print(f"Progreso: {i:,}/{total_pairs:,} ({progress:.1f}%)")
    
    return cooccurrence_matrix.tocsr(), product_to_idx, idx_to_product

def get_cooccurrence_recommendations(cooccurrence_matrix, product_to_idx, idx_to_product, 
                                   product_id, n_recommendations=10, weight=0.3):
    """
    Genera recomendaciones basadas en co-ocurrencia
    """
    if product_id not in product_to_idx:
        print(f"Producto {product_id} no encontrado en matriz")
        return pd.DataFrame()
    
    product_idx = product_to_idx[product_id]
    scores = cooccurrence_matrix[product_idx].toarray().flatten()
    
    # Obtener índices de productos recomendados (excluyendo el mismo producto)
    scores[product_idx] = 0
    top_indices = np.argsort(scores)[::-1][:n_recommendations]
    
    recommendations = []
    for idx in top_indices:
        if scores[idx] > 0:
            recommendations.append({
                'producto': idx_to_product[idx],
                'cooccurrence_score': scores[idx] * weight
            })
    
    return pd.DataFrame(recommendations)

# Construir matriz de co-ocurrencia
print("Construyendo matriz de co-ocurrencia...")
cooccurrence_matrix, product_to_idx, idx_to_product = build_cooccurrence_matrix_optimized(df_b2b_enhanced, co_frequency)

# Probar con un producto del cliente
client_products = df_b2b_enhanced[df_b2b_enhanced['id_b2b'] == test_client]['producto'].unique()
test_product = client_products[0] if len(client_products) > 0 else 'Producto_377'

print(f"Generando recomendaciones para {test_product}...")
cooccurrence_recs = get_cooccurrence_recommendations(cooccurrence_matrix, product_to_idx, idx_to_product, 
                                                   test_product, 5)
print(f"Recomendaciones de co-ocurrencia para {test_product}:")
print(cooccurrence_recs)

Construyendo matriz de co-ocurrencia...
=== VERSIÓN OPTIMIZADA ===
Precalculando valores promedio por producto...
Valores promedio calculados para 2,564 productos
Procesando 1,156,241 pares...
Progreso: 0/1,156,241 (0.0%)
Progreso: 10,000/1,156,241 (0.9%)
Progreso: 20,000/1,156,241 (1.7%)
Progreso: 30,000/1,156,241 (2.6%)
Progreso: 40,000/1,156,241 (3.5%)
Progreso: 50,000/1,156,241 (4.3%)
Progreso: 60,000/1,156,241 (5.2%)
Progreso: 70,000/1,156,241 (6.1%)
Progreso: 80,000/1,156,241 (6.9%)
Progreso: 90,000/1,156,241 (7.8%)
Progreso: 100,000/1,156,241 (8.6%)
Progreso: 110,000/1,156,241 (9.5%)
Progreso: 120,000/1,156,241 (10.4%)
Progreso: 130,000/1,156,241 (11.2%)
Progreso: 140,000/1,156,241 (12.1%)
Progreso: 150,000/1,156,241 (13.0%)
Progreso: 160,000/1,156,241 (13.8%)
Progreso: 170,000/1,156,241 (14.7%)
Progreso: 180,000/1,156,241 (15.6%)
Progreso: 190,000/1,156,241 (16.4%)
Progreso: 200,000/1,156,241 (17.3%)
Progreso: 210,000/1,156,241 (18.2%)
Progreso: 220,000/1,156,241 (19.0%)
Progre

## Sistema Híbrido B2B Final

In [13]:
def hybrid_b2b_recommendation_system(df, client_metrics, cliente_producto_stats, 
                                   producto_zona_stats, cooccurrence_matrix, 
                                   product_to_idx, idx_to_product, client_id, 
                                   n_recommendations=10, exclude_recent_days=30):
    """
    Sistema híbrido de recomendaciones B2B que combina 4 componentes:
    1. Maximización de Valor (30%)
    2. Predictor de Reposición (40%) 
    3. Cross-Selling Geográfico (20%)
    4. Co-ocurrencia Inteligente (10%)
    """
    
    # Pesos para cada componente
    weights = {
        'value': 0.3,
        'replenishment': 0.4,
        'geographic': 0.2,
        'cooccurrence': 0.1
    }
    
    print(f"=== GENERANDO RECOMENDACIONES HÍBRIDAS PARA {client_id} ===")
    
    # Obtener productos que el cliente ha comprado recientemente (para filtrar)
    recent_products = set()
    if len(df[df['id_b2b'] == client_id]) > 0:
        fecha_limite = df['fecha'].max() - pd.Timedelta(days=exclude_recent_days)
        recent_products = set(df[(df['id_b2b'] == client_id) & 
                               (df['fecha'] >= fecha_limite)]['producto'].unique())
    
    # 1. COMPONENTE DE VALOR
    print("Calculando scores de valor...")
    value_recs = get_value_recommendations(df, client_id, n_recommendations=50)
    value_scores = dict(zip(value_recs['producto'], value_recs['value_score'] * weights['value']))
    
    # 2. COMPONENTE DE REPOSICIÓN  
    print("Calculando scores de reposición...")
    replenishment_recs = get_replenishment_recommendations(df, cliente_producto_stats, client_id, n_recommendations=50)
    replenishment_scores = dict(zip(replenishment_recs['producto'], 
                                  replenishment_recs['replenishment_score'] * weights['replenishment']))
    
    # 3. COMPONENTE GEOGRÁFICO
    print("Calculando scores geográficos...")
    geographic_recs = get_geographic_recommendations(df, producto_zona_stats, client_id, n_recommendations=50)
    geographic_scores = dict(zip(geographic_recs['producto'], 
                               geographic_recs['geographic_score'] * weights['geographic']))
    
    # 4. COMPONENTE DE CO-OCURRENCIA
    print("Calculando scores de co-ocurrencia...")
    cooccurrence_scores = {}
    
    # Obtener productos que el cliente ya compró para generar co-ocurrencias
    client_products = df[df['id_b2b'] == client_id]['producto'].unique()
    
    for client_product in client_products[:5]:  # Limitamos a 5 productos por eficiencia
        co_recs = get_cooccurrence_recommendations(cooccurrence_matrix, product_to_idx, 
                                                 idx_to_product, client_product, n_recommendations=20)
        for _, row in co_recs.iterrows():
            producto = row['producto']
            score = row['cooccurrence_score'] * weights['cooccurrence']
            if producto in cooccurrence_scores:
                cooccurrence_scores[producto] = max(cooccurrence_scores[producto], score)
            else:
                cooccurrence_scores[producto] = score
    
    # 5. COMBINACIÓN HÍBRIDA
    print("Combinando scores...")
    all_products = set(value_scores.keys()) | set(replenishment_scores.keys()) | \
                   set(geographic_scores.keys()) | set(cooccurrence_scores.keys())
    
    hybrid_recommendations = []
    
    for producto in all_products:
        # Saltar productos comprados recientemente
        if producto in recent_products:
            continue
            
        # Combinar scores (algunos productos pueden no tener todos los scores)
        final_score = (
            value_scores.get(producto, 0) +
            replenishment_scores.get(producto, 0) +
            geographic_scores.get(producto, 0) +
            cooccurrence_scores.get(producto, 0)
        )
        
        # Calcular scores individuales para análisis
        individual_scores = {
            'valor': value_scores.get(producto, 0),
            'reposicion': replenishment_scores.get(producto, 0),
            'geografico': geographic_scores.get(producto, 0),
            'coocurrencia': cooccurrence_scores.get(producto, 0)
        }
        
        # Solo incluir si tiene al menos un score > 0
        if final_score > 0:
            hybrid_recommendations.append({
                'producto': producto,
                'score_hibrido': final_score,
                'score_valor': individual_scores['valor'],
                'score_reposicion': individual_scores['reposicion'],
                'score_geografico': individual_scores['geografico'],
                'score_coocurrencia': individual_scores['coocurrencia'],
                'componentes_activos': sum(1 for score in individual_scores.values() if score > 0)
            })
    
    # Ordenar por score final y tomar top N
    hybrid_recommendations = sorted(hybrid_recommendations, key=lambda x: x['score_hibrido'], reverse=True)
    hybrid_recommendations = hybrid_recommendations[:n_recommendations]
    
    # Convertir a DataFrame
    recommendations_df = pd.DataFrame(hybrid_recommendations)
    
    # Añadir información adicional del producto
    if len(recommendations_df) > 0:
        product_info = df.groupby('producto').agg({
            'valor_total': 'mean',
            'alineación con portafolio estratégico b2b': 'mean',
            'categoria_b2b_macro': 'first'
        }).round(2)
        
        recommendations_df = recommendations_df.merge(
            product_info, left_on='producto', right_index=True, how='left'
        )
        
        recommendations_df.rename(columns={
            'valor_total': 'valor_promedio',
            'alineación con portafolio estratégico b2b': 'alineacion_estrategica',
            'categoria_b2b_macro': 'categoria'
        }, inplace=True)
    
    return recommendations_df

def analyze_recommendation_composition(recommendations_df):
    """
    Analiza la composición de las recomendaciones híbridas
    """
    if len(recommendations_df) == 0:
        print("No hay recomendaciones para analizar")
        return
    
    print("\n=== ANÁLISIS DE COMPOSICIÓN DE RECOMENDACIONES ===")
    
    # Distribución por número de componentes activos
    print("Distribución por componentes activos:")
    comp_dist = recommendations_df['componentes_activos'].value_counts().sort_index()
    for componentes, count in comp_dist.items():
        percentage = (count / len(recommendations_df)) * 100
        print(f"  {componentes} componentes: {count} productos ({percentage:.1f}%)")
    
    # Contribución promedio por componente
    print("\nContribución promedio por componente:")
    for comp in ['score_valor', 'score_reposicion', 'score_geografico', 'score_coocurrencia']:
        avg_score = recommendations_df[comp].mean()
        print(f"  {comp.replace('score_', '').title()}: {avg_score:.4f}")
    
    # Top productos por categoría
    print("\nTop productos por categoría:")
    if 'categoria' in recommendations_df.columns:
        cat_dist = recommendations_df['categoria'].value_counts().head(3)
        for categoria, count in cat_dist.items():
            print(f"  {categoria}: {count} productos")



In [14]:
# EJEMPLO DE USO COMPLETO
def ejemplo_sistema_hibrido():
    """
    Ejemplo completo de uso del sistema híbrido
    """
    # Seleccionar cliente de prueba
    test_client = df_b2b_enhanced['id_b2b'].unique()[0]
    
    print(f"=== EJEMPLO SISTEMA HÍBRIDO B2B ===")
    print(f"Cliente: {test_client}")
    
    # Información del cliente
    client_info = client_metrics[client_metrics['id_b2b'] == test_client].iloc[0]
    print(f"Segmento: {client_info['segmento_valor']}")
    print(f"Valor total histórico: ${client_info['valor_total_cliente']:,.0f}")
    print(f"Transacciones: {client_info['n_transacciones_cliente']}")
    
    # Generar recomendaciones híbridas
    hybrid_recs = hybrid_b2b_recommendation_system(
        df_b2b_enhanced, client_metrics, cliente_producto_stats, 
        producto_zona_stats, cooccurrence_matrix, product_to_idx, 
        idx_to_product, test_client, n_recommendations=3
    )
    
    print(f"\n=== TOP 10 RECOMENDACIONES HÍBRIDAS ===")
    if len(hybrid_recs) > 0:
        display_cols = ['producto', 'score_hibrido', 'valor_promedio', 'alineacion_estrategica', 
                       'componentes_activos', 'categoria']
        print(hybrid_recs[display_cols].round(4))
        
        # Análisis de composición
        analyze_recommendation_composition(hybrid_recs)
        
        # Desglose del top 3
        print(f"\n=== DESGLOSE TOP 3 RECOMENDACIONES ===")
        for i, row in hybrid_recs.head(3).iterrows():
            print(f"\n{i+1}. {row['producto']} (Score: {row['score_hibrido']:.4f})")
            print(f"   Valor: {row['score_valor']:.4f} | Reposición: {row['score_reposicion']:.4f}")
            print(f"   Geográfico: {row['score_geografico']:.4f} | Co-ocurrencia: {row['score_coocurrencia']:.4f}")
            print(f"   Categoría: {row.get('categoria', 'N/A')} | Valor promedio: ${row.get('valor_promedio', 0):,.0f}")
    else:
        print("No se generaron recomendaciones para este cliente")
    
    return hybrid_recs

# EJECUTAR EJEMPLO
resultado_ejemplo = ejemplo_sistema_hibrido()

=== EJEMPLO SISTEMA HÍBRIDO B2B ===
Cliente: B2B_01
Segmento: Medio_Valor
Valor total histórico: $1,916,077
Transacciones: 2142
=== GENERANDO RECOMENDACIONES HÍBRIDAS PARA B2B_01 ===
Calculando scores de valor...
Calculando scores de reposición...
Calculando scores geográficos...
Calculando scores de co-ocurrencia...
Combinando scores...

=== TOP 10 RECOMENDACIONES HÍBRIDAS ===
       producto  score_hibrido  valor_promedio  alineacion_estrategica  \
0  Producto_377        19.2532         3659.24                     0.0   
1  Producto_190        10.2863         2231.49                     0.0   
2  Producto_884         8.6351         4232.65                     0.0   

   componentes_activos         categoria  
0                    3  cat_b2b_macro_10  
1                    3   cat_b2b_macro_3  
2                    2  cat_b2b_macro_10  

=== ANÁLISIS DE COMPOSICIÓN DE RECOMENDACIONES ===
Distribución por componentes activos:
  2 componentes: 1 productos (33.3%)
  3 componentes: 2 prod

In [15]:
def generar_recomendaciones_4_modelos(df, client_metrics, cliente_producto_stats, 
                                     producto_zona_stats, cooccurrence_matrix, 
                                     product_to_idx, idx_to_product, client_id, 
                                     n_recommendations=10):
    """
    Genera recomendaciones de los 4 modelos básicos de forma simple
    
    Returns:
        dict: Diccionario con 4 DataFrames: valor, reposicion, geografico, coocurrencia
    """
    
    resultados = {}
    
    # 1. MODELO DE VALOR
    valor_recs = get_value_recommendations(df, client_id, n_recommendations)
    if len(valor_recs) > 0:
        df_valor = valor_recs[['producto', 'value_score']].copy()
        df_valor.rename(columns={'value_score': 'score'}, inplace=True)
        df_valor['ranking'] = range(1, len(df_valor) + 1)
    else:
        df_valor = pd.DataFrame(columns=['producto', 'score', 'ranking'])
    resultados['valor'] = df_valor
    # 2. MODELO DE REPOSICIÓN
    reposicion_recs = get_replenishment_recommendations(df, cliente_producto_stats, client_id, n_recommendations)
    if len(reposicion_recs) > 0:
        df_reposicion = reposicion_recs[['producto', 'replenishment_score']].copy()
        df_reposicion.rename(columns={'replenishment_score': 'score'}, inplace=True)
        df_reposicion['ranking'] = range(1, len(df_reposicion) + 1)
    else:
        df_reposicion = pd.DataFrame(columns=['producto', 'score', 'ranking'])
    resultados['reposicion'] = df_reposicion
    
    # 3. MODELO GEOGRÁFICO
    geografico_recs = get_geographic_recommendations(df, producto_zona_stats, client_id, n_recommendations)
    if len(geografico_recs) > 0:
        df_geografico = geografico_recs[['producto', 'geographic_score']].copy()
        df_geografico.rename(columns={'geographic_score': 'score'}, inplace=True)
        df_geografico['ranking'] = range(1, len(df_geografico) + 1)
    else:
        df_geografico = pd.DataFrame(columns=['producto', 'score', 'ranking'])
    resultados['geografico'] = df_geografico
    
    # 4. MODELO DE CO-OCURRENCIA
    client_products = df[df['id_b2b'] == client_id]['producto'].unique()
    if len(client_products) > 0:
        all_cooccurrence = []
        for client_product in client_products[:3]:
            co_recs = get_cooccurrence_recommendations(cooccurrence_matrix, product_to_idx, 
                                                     idx_to_product, client_product, n_recommendations=20)
            if len(co_recs) > 0:
                all_cooccurrence.append(co_recs)
        
        if all_cooccurrence:
            combined_co = pd.concat(all_cooccurrence, ignore_index=True)
            df_coocurrencia = combined_co.groupby('producto')['cooccurrence_score'].max().reset_index()
            df_coocurrencia = df_coocurrencia.sort_values('cooccurrence_score', ascending=False).head(n_recommendations)
            df_coocurrencia.rename(columns={'cooccurrence_score': 'score'}, inplace=True)
            df_coocurrencia['ranking'] = range(1, len(df_coocurrencia) + 1)
        else:
            df_coocurrencia = pd.DataFrame(columns=['producto', 'score', 'ranking'])
    else:
        df_coocurrencia = pd.DataFrame(columns=['producto', 'score', 'ranking'])
    resultados['coocurrencia'] = df_coocurrencia
    
    return resultados

In [16]:
# Ejemplo de uso
cliente_test = df_b2b_enhanced['id_b2b'].unique()[0]

# Generar recomendaciones
recomendaciones = generar_recomendaciones_4_modelos(
    df_b2b_enhanced, client_metrics, cliente_producto_stats, 
    producto_zona_stats, cooccurrence_matrix, product_to_idx, 
    idx_to_product, cliente_test, n_recommendations=5
)


In [17]:
recomendaciones['valor']




,producto,score,ranking
891,Producto_983,4.151230,1
1146,Producto_1247,3.767888,2
1052,Producto_1151,2.924689,3
1147,Producto_1248,2.757262,4
890,Producto_982,2.427475,5


In [18]:
recomendaciones['reposicion']

,producto,score,ranking
543,Producto_54,0.45,1
480,Producto_377,0.45,2
586,Producto_68,0.45,3
474,Producto_366,0.45,4
584,Producto_67,0.45,5


In [19]:
recomendaciones['geografico']


,producto,score,ranking
1547,Producto_377,0.195109,1
610,Producto_167,0.059718,2
1598,Producto_438,0.055995,3
1321,Producto_251,0.054715,4
1792,Producto_721,0.053717,5


In [20]:
recomendaciones['coocurrencia']

,producto,score,ranking
25,Producto_377,190.342269,1
13,Producto_167,104.116392,2
18,Producto_190,101.901482,3
14,Producto_179,91.962454,4
35,Producto_61,90.309943,5


## Funcion de recomendacion Hibrida

In [21]:
def generar_recomendaciones_hibridas_rrf(df, client_metrics, cliente_producto_stats, 
                                       producto_zona_stats, cooccurrence_matrix, 
                                       product_to_idx, idx_to_product, client_id, 
                                       n_recommendations=10):
    """
    Genera recomendaciones híbridas usando Reciprocal Rank Fusion (RRF)
    
    Returns:
        tuple: (recomendaciones_hibridas_df, modelos_individuales_dict)
    """
    
    pesos = {
        'valor': 0.0,        # Maximización de valor
        'reposicion': 0.15,   # Predicción de reposición (más peso por ser B2B)
        'geografico': 0.65,   # Cross-selling geográfico  
        'coocurrencia': 0.20  # Co-ocurrencia (menor peso por ser más exploratorio)
    }
    
    

    
    # 1. Obtener recomendaciones individuales
    modelos_individuales = generar_recomendaciones_4_modelos(
        df, client_metrics, cliente_producto_stats, 
        producto_zona_stats, cooccurrence_matrix, 
        product_to_idx, idx_to_product, client_id, 
        n_recommendations=50  # Más productos para mayor diversidad
    )
    
    # 2. Crear diccionario de rankings por producto para cada modelo
    rankings_por_modelo = {}
    
    for modelo, df_modelo in modelos_individuales.items():
        if len(df_modelo) > 0:
            rankings_por_modelo[modelo] = dict(zip(df_modelo['producto'], df_modelo['ranking']))
        else:
            rankings_por_modelo[modelo] = {}
    
    # 3. Obtener todos los productos únicos recomendados
    todos_productos = set()
    for modelo_df in modelos_individuales.values():
        if len(modelo_df) > 0:
            todos_productos.update(modelo_df['producto'].tolist())
    
    # 4. Calcular RRF Score para cada producto
    k = 60  # Parámetro RRF (típicamente entre 60-100)
    
    productos_scores = []
    
    for producto in todos_productos:
        # RRF Score = suma de peso_modelo / (k + ranking_modelo)
        rrf_score = 0.0
        rankings_individuales = {}
        modelos_activos = 0
        
        for modelo in ['valor', 'reposicion', 'geografico', 'coocurrencia']:
            if producto in rankings_por_modelo[modelo]:
                ranking = rankings_por_modelo[modelo][producto]
                rrf_score += pesos[modelo] / (k + ranking)
                rankings_individuales[f'rank_{modelo}'] = ranking
                modelos_activos += 1
            else:
                rankings_individuales[f'rank_{modelo}'] = None
        
        # Solo incluir productos recomendados por al menos 1 modelo
        if modelos_activos > 0:
            productos_scores.append({
                'producto': producto,
                'score_hibrido_rrf': rrf_score,
                'rank_valor': rankings_individuales.get('rank_valor'),
                'rank_reposicion': rankings_individuales.get('rank_reposicion'),
                'rank_geografico': rankings_individuales.get('rank_geografico'),
                'rank_coocurrencia': rankings_individuales.get('rank_coocurrencia'),
                'modelos_activos': modelos_activos
            })
    
    # 5. Ordenar por score RRF y asignar ranking híbrido
    productos_scores.sort(key=lambda x: x['score_hibrido_rrf'], reverse=True)
    
    for i, producto_info in enumerate(productos_scores[:n_recommendations]):
        producto_info['ranking_hibrido'] = i + 1
    
    # 6. Crear DataFrame final
    recomendaciones_hibridas = pd.DataFrame(productos_scores[:n_recommendations])
    
    return recomendaciones_hibridas, modelos_individuales

In [22]:
# EJEMPLO DE USO COMPLETO DEL SISTEMA HÍBRIDO

# Seleccionar cliente para prueba
cliente_prueba = df_b2b_enhanced['id_b2b'].unique()[1]  # Cambiar índice si quieres otro cliente

print(f"=== DEMO SISTEMA DE RECOMENDACIONES HÍBRIDO ===")
print(f"Cliente seleccionado: {cliente_prueba}")

# Información del cliente
if cliente_prueba in client_metrics['id_b2b'].values:
    info_cliente = client_metrics[client_metrics['id_b2b'] == cliente_prueba].iloc[0]
    print(f"Segmento: {info_cliente['segmento_valor']}")
    print(f"Valor total histórico: ${info_cliente['valor_total_cliente']:,.0f}")
    print(f"Número de transacciones: {info_cliente['n_transacciones_cliente']:,}")
    print(f"Productos distintos comprados: {info_cliente['productos_distintos_cliente']}")

# Generar recomendaciones híbridas
recomendaciones_hibridas, modelos_individuales = generar_recomendaciones_hibridas_rrf(
    df_b2b_enhanced, client_metrics, cliente_producto_stats, 
    producto_zona_stats, cooccurrence_matrix, product_to_idx, 
    idx_to_product, cliente_prueba, n_recommendations=15
)
recomendaciones_hibridas

=== DEMO SISTEMA DE RECOMENDACIONES HÍBRIDO ===
Cliente seleccionado: B2B_02
Segmento: Alto_Valor
Valor total histórico: $8,624,149
Número de transacciones: 6,583
Productos distintos comprados: 1361


,producto,score_hibrido_rrf,rank_valor,rank_reposicion,rank_geografico,rank_coocurrencia,modelos_activos,ranking_hibrido
0,Producto_1569,0.014534,None,19.0,8,5.0,3,1
1,Producto_157,0.014071,None,46.0,4,20.0,3,2
2,Producto_167,0.013882,None,NaN,1,2.0,2,3
3,Producto_1480,0.013785,None,36.0,3,45.0,3,4
4,Producto_226,0.012588,None,13.0,16,41.0,3,5
5,Producto_73,0.012484,None,NaN,2,40.0,2,6
6,Producto_377,0.012306,None,NaN,12,1.0,2,7
7,Producto_1479,0.012105,None,NaN,5,35.0,2,8
8,Producto_910,0.011468,None,NaN,13,18.0,2,9
9,Producto_75,0.010749,None,5.0,17,NaN,2,10


## Función para cargar modelos entrenado

In [23]:
import os

def exportar_modelos(path_destino):
    """
    Exporta los modelos entrenados al path especificado
    """
    # Crear directorio si no existe
    os.makedirs(path_destino, exist_ok=True)
    
    print(f"Exportando modelos a: {path_destino}")
    
    try:
        # Exportar componentes esenciales
        client_metrics.to_pickle(f"{path_destino}/client_metrics.pkl")
        cliente_producto_stats.to_pickle(f"{path_destino}/cliente_producto_stats.pkl")
        producto_zona_stats.to_pickle(f"{path_destino}/producto_zona_stats.pkl")
        
        # Exportar matriz de co-ocurrencia
        pd.DataFrame(cooccurrence_matrix.toarray()).to_pickle(f"{path_destino}/cooccurrence_matrix.pkl")
        
        # Exportar mapeos de productos
        pd.Series(product_to_idx).to_pickle(f"{path_destino}/product_to_idx.pkl")
        pd.Series(idx_to_product).to_pickle(f"{path_destino}/idx_to_product.pkl")
        
        # Exportar DataFrame principal
        df_b2b_enhanced.to_pickle(f"{path_destino}/df_b2b_enhanced.pkl")
        
        print("Modelos exportados correctamente")
        
    except Exception as e:
        print(f"Error exportando: {str(e)}")

## Cargar modelos

In [24]:
PATH_MODELOS = "../Modelos/ModelosCompletos"

# Exportar todos los modelos
exportar_modelos(PATH_MODELOS)

Exportando modelos a: ../Modelos/ModelosCompletos
Modelos exportados correctamente
